In [1]:
import os
import pickle
import string
import argparse

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1

REBUILDDICTIONARYFROMPICKLEFILE = 0

path = 'F:\LaTeX\LatexWork\Bible'
os.chdir(path)
print (os.getcwd())

import plotly

F:\LaTeX\LatexWork\Bible


In [2]:
# %load BibleWordList2.py
'''
Update 21 Deceber 2018, 0004
'''


import os
import pickle
import collections
import operator
import re

DEBUG = 0

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1
REBUILDDICTIONARYFROMPICKLEFILE = 0

punctuations = '''!()_;:,<>?@#$^.'''

exceptions = [
    'A', 'Accept', 'After', 'Aha', 'All','Alas','Also','Arise',
    'Amen', 'An', 'And', 'As', 'At', 'Awake','According',
    'Be', 'Because', 'Before', 'Behold', 'Blessed','Build','Better',
    'Blessings', 'Blind', 'Both', 'Bring', 'But','Beautiful',
    'By', 'Command', 'Concerning', 'Consider', 'Cut','Come',"Children's",
    'Deal', 'Deceit', 'Deep', 'Deliver', 'Depart','Did',
    'Doubtless', 'Each', 'Either', 'Even', 'Every','Excellent','Fear',
    'For', 'From', 'Get', 'Give', 'Go', 'Grace',
    'Great', 'Hatred', 'Having', 'He', 'Hear',
    'Hearing', 'Heaviness', 'Help', 'His', 'Hold',
    'Horror', 'How', 'If', 'In', 'Incline', 'Judge',
    'Leave', 'Let', 'Lo', 'Look', 'Lying', 'Lodge','Make','Must','Mark',
    'Many', 'Mine', 'My', 'Neither', 'Nevertheless','Nay',
    'Not', 'Now', 'Number', 'O', 'Of', 'On', 'One','Out',
    'Only', 'Open', 'Or', 'Order', 'Our', 'Pardon',
    'Plead', 'Princes', 'Quicken', 'Red', 'Remember',
    'Remove', 'Riches', 'Righteous', 'Rise', 'Rivers','Return',
    'Sacrifice', 'Salvation', 'Say', 'Seven', 'Shall','Spake',
    'So', 'Speak', 'Sprinkle', 'Stablish', 'Stand','Strong',
    'Surely', 'Take', 'Teach', 'That', 'The', 'Their',
    'Then', 'There', 'Therefore', 'These', 'They',
    'This', 'Those', 'Thou', 'Though', 'Through',
    'Thus', 'Thy', 'To', 'Treasures', 'Trouble','Told',
    'Turn', 'Two', 'Unless', 'Uphold', 'We',
    'What', 'Whatsoever', 'When', 'Where', 'Wherefore',
    'Wherewithal', 'Which', 'Who', 'Whom', 'Whoso',
    'Whosoever', 'Why', 'Wise', 'With', 'Withhold','Walk','Wisdom',
    'Would', 'Ye', 'Yea', 'Yet', 'Your']

properNameList = [
    'Aaron', 'Abelmizraim', 'Abiasaph', 'Abib', 'Abihu', 'Abraham', 'Abram',
    'Aceldama', 'Achaia', 'Achaicus', 'Achor', 'Achzib', 'Adam', 'Adramyttium',
    'Adria', 'Adullam', 'Agrippa', 'Ahab', 'Ahaz', 'Ahikam', 'Ahisamach',
    'Aholiab', 'Alexandria', 'Alexandrians', 'Alphaeus', 'Amalek', 'Amittai',
    'Ammi ', 'Amminadab', 'Amon', 'Amorite', 'Amorites', 'Amram', 'Anathoth',
    'Andrew', 'Angel', 'Antioch', 'Aphrah', 'Apollos', 'Apphia', 'Aquila',
    'Arabia ', 'Arabian', 'Archippus', 'Ard', 'Areli', 'Aretas', 'Aristarchus',
    'Arnon', 'Arodi', 'Aroer ', 'Asenath', 'Asenath', 'Ashbel', 'Asher',
    'Ashkelon', 'Asia', 'Assir', 'Assyria ', 'Assyrian', 'Atad', 'Augustus',
    'Azariah', 'Azur', 'BRANCH', 'Baal ', 'Baali ', 'Baalim', 'Baalzephon',
    'Bablyon', 'Balaam', 'Balak', 'Barbarian', 'Barjona', 'Barnabas',
    'Barsabas', 'Bartholomew', 'Baruch', 'Becher', 'Beeri', 'Beerlahairoi',
    'Beersheba', 'Belah', 'Belial', 'Belteshazzar', 'Benjamin', 'Beor',
    'Bered', 'Beriah', 'Bernice', 'Bethaven', 'Bethdiblathaim ', 'Bethel',
    'Bethezel', 'Bethgamul', 'Bethlehem', 'Bethmeon', 'Bethshemesh',
    'Bezaleel', 'Bilhah', 'Bozrah', 'Caesar ', 'Caesarea', 'Cainan', 'Cana',
    'Canaan', 'Canaanite', 'Canaanites', 'Canaanitish', 'Capernaum', 'Caphtor',
    'Carchemish', 'Carmi', 'Castor', 'Cephas', 'Chaldeans', 'Chaldeans',
    'Chemosh', 'Chittim', 'Chloe', 'Christ', 'Cilicia', 'Clauda', 'Cnidus',
    'Crete', 'Cretians', 'Crispus', 'Cyprus', 'Cyrenians', 'Cyrus',
    'Damascenes', 'Damascus', 'Dan', 'Daniel', 'Darius', 'David', 'Demas',
    'Diblaim', 'Dibon', 'Didymus', 'Dinah', 'Edom', 'Egypt', 'Egyptian',
    'Egyptians', 'Ehi', 'Elasah', 'Eleazar', 'Elias', 'Eliezer', 'Elim',
    'Elisheba', 'Elkanah', 'Elon', 'Elzaphan', 'Enoch', 'Enos', 'Epaphras',
    'Ephesus', 'Ephraim', 'Ephrath', 'Ephron', 'Er', 'Eri', 'Esaias', 'Esau',
    'Etham', 'Euphrates', 'Euroclydon', 'Ezbon', 'Father', 'Felix', 'Festus',
    'Fortunatus', 'Gad', 'Gadarenes', 'Gaius', 'Galatia', 'Galilee', 'Gath',
    'Gaza', 'Gedaliah', 'Gemariah', 'Gentiles', 'Gera', 'Gershom', 'Gershon',
    'Gibeah', 'Gibeon', 'Gilead', 'Gilgal', 'God', 'Gomer', 'Goshen',
    'Grecians', 'Greek', 'Guni', 'Hagar', 'Haggi', 'Ham', 'Hananiah', 'Hanoch',
    'Heber', 'Hebrew', 'Hebrews', "Hebrews'", 'Hebron', 'Heldai', 'Helem',
    'Hen', 'Heshbon', 'Hezekiah', 'Hezron', 'High', 'Hilkiah', 'Hinnom',
    'Hittite', 'Hittites', 'Hivite', 'Hivites', 'Horeb', 'Horonaim', 'Hosea',
    'Hoshaiah', 'Huppim', 'Hur', 'Hushim', 'I', 'Isaac', 'Ishi', 'Ishmael',
    'Ishmeelites', 'Ishuah', 'Israel', 'Issachar', 'Issachar', 'Isui', 'Italy',
    'Ithamar', 'Ithiel', 'Izhar', 'JEHOVAH', 'Jachin', 'Jacob', 'Jahleel',
    'Jahzeel', 'Jairus', 'Jakeh', 'James', 'Jamin', 'Japheth', 'Jareb:',
    'Jared', 'Jebusite', 'Jebusites', 'Jeconiah', 'Jedaiah', 'Jehoiada',
    'Jehoiakim', 'Jehoshapha', 'Jehovahnissi', 'Jehu', 'Jemuel', 'Jeremiah',
    'Jerusalem', 'Jesus', 'Jethro', 'Jews', 'Jezer', 'Jezreel', 'Jimnah',
    'Joash', 'Job', 'Jochebed', 'Joel', 'Johanan', 'John', 'Jonah', 'Jonas',
    'Joppa', 'Jordan', 'Josedech', 'Joseph', 'Josiah', 'Jotham', 'Judaea',
    'Judah', 'Judas', 'Julius', 'Just ', 'Justus ', 'Kadesh', 'Kareah',
    'Kedar', 'Kerioth', 'King', 'Kiriathaim ', 'Kohath', 'Korah', 'Korhites',
    'LORD', 'Laban', 'Laban', 'Lachis', 'Lamech', 'Laodicea', 'Lasea', 'Leah',
    'Lebanon', 'Legion', 'Levi', 'Libertines', 'Libni', 'Libyans', 'Loammi',
    'Lord', 'Lord Jesus Christ', 'Loruhamah', 'Lucas', 'Luhith', 'Luz', 'Lycia',
    'Lydians', 'Maaseiah', 'Macedonia', 'Macedonian', 'Machir', 'Machpelah',
    'Mahalaleel', 'Mahali', 'Malachi', 'Malchiel', 'Mamre', 'Manasseh',
    'Marcus', 'Mareshah', 'Maroth', 'Massah', 'Matthew', 'Matthias',
    'Melchizedek', 'Melita', 'Melzar', 'Merari', 'Meribah', 'Messias',
    'Methuselah', 'Micah', 'Midian', 'Migdol', 'Miriam', 'Misgab', 'Mishael',
    'Mizpah', 'Moab', 'Morasthite', 'Moreshethgath', 'Moses', 'Muppim', 'Mushi',
    'Myra', 'Naaman', 'Naashon', 'Nadab', 'Naphtali', 'Nathanael', 'Nazareth',
    'Nebo', 'Nebuchadnezzar', 'Nebuchadrezzar', 'Nebuzaradan', 'Nehelamite',
    'Nepheg', 'Neriah', 'Nicanor', 'Nicolas', 'Nicopolis', 'Nineveh', 'No ',
    'Noah', 'Noph', 'Ohad', 'Olivet', 'Omri', 'Onan', 'Onesimus', 'Padan',
    'Padanaram', 'Palestine', 'Pallu', 'Pamphylia', 'Parmenas', 'Pashur',
    'Pathros', 'Paul', 'Pentecost', 'Pergamos', 'Perizzite', 'Perizzites',
    'Peter', 'Pethuel', 'Phallu', 'Pharaoh', "Pharaoh's", 'Pharaohhophra ',
    'Pharaohnecho', 'Pharez', 'Pharisee ', 'Pharisees', 'Phenice',
    'Philadelphia', 'Philemon', 'Philip', 'Philippi', 'Philistines',
    'Phinehas', 'Phuvah', 'Pihahiroth', 'Pilate', 'Pithom', 'Pollux',
    'Potiphar', 'Potipherah', 'Prince', 'Priscilla', 'Prochorus', 'Puah',
    'Publius', 'Puteoli', 'Putiel', 'Raamses', 'Rachel', 'Ramah', 'Rameses',
    'Rephidim', 'Reuben', 'Reuel', 'Rhegium', 'Rock', 'Rome', 'Rosh', 'Ruhamah',
    'Sabeans', 'Sadducees', 'Salem', 'Salmone', 'Samaria', 'Samaritans',
    'Samuel', 'Saphir', 'Sara', 'Sarah', 'Sarai', 'Sardis', 'Satan', 'Saul',
    'Saviour', 'Scythian', 'Selah', 'Sepharad', 'Serah', 'Sered', 'Seth',
    'Shaphan', 'Shaul', 'Shelah', 'Shem', 'Shemaiah', 'Shillem', 'Shiloh',
    'Shimi', 'Shimron', 'Shiphrah', 'Shittim', 'Shuni', 'Sidon', 'Sihon',
    'Sihor', 'Simeon', 'Simon ', 'Simon Zelotes', 'Sinai', 'Sirion', 'Smyrna',
    'Solomon', 'Sosthenes', 'Stephanas', 'Stephen', 'Succoth', 'Sychar',
    'Syracuse', 'Tabor', 'Tahpanhes', 'Tarshish', 'Teman', 'Theophilus',
    'Thessalonica', 'Thomas', 'Thummim', 'Thyatira', 'Tiberias', 'Timon',
    'Timotheus', 'Timothy', 'Titus', 'Tobijah', 'Tola', 'Tophet', 'Tychicus',
    'Tyre', 'Tyrus', 'Ucal', 'Uri', 'Urim', 'Uzziah', 'Uzziel', 'Zaanan',
    'Zaphnathpaaneah', 'Zarephath', 'Zebedee', 'Zebulun', 'Zedekiah',
    'Zephaniah', 'Zerah', 'Zichri', 'Zidon', 'Zilpah', 'Zion', 'Ziphion',
    'Zipporah', 'Zithri', 'Zohar']

##########
def removePunc(input):
    no_punc = ''
    for char in input:
        if char not in punctuations:
            no_punc = no_punc + char
    return no_punc


##########
def generatedLatexFormattedText(dict):
    BUILDAWIP = 1
    BUILDNWIV = 1
    BUILDPNIP = 1
    BUILDNWIC = 1
    BUILDNLIW = 1
    BUILDBPIP = 1
    MINISTRY = 1

    topString = "\\footnote{\\textcolor[cmyk]{0.99998,1,0,0}{\\hyperlink{TOC}{Return to end " \
                "of Table of Contents.}}}"
    textColorString = "\\textcolor[cmyk]{0.99998,1,0,0}{"
    endString = "}"

    '''
    Print plain text with Latex color header:
        \textcolor[rgb]{0.00,0.00,1.00} == BLUE
        \textcolor[cmyk]{0.99998,1,0,0} == A DEEPER BLUE
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with Latex format\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')

    linesWith13Words = []
    linesWith18Words = []
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        newline = line[5]
        if versn2 == '1':
            print(topString + textColorString + newline + endString)
        else:
            print('[' + versn2 + '] ' + textColorString + newline + endString)
        words = newline.split()
        wordLength = len(words)
        if wordLength == 13:
            linesWith13Words.append(line)
        if wordLength == 18:
            linesWith18Words.append(line)
            
        
    '''
    NOW print the verses with LaTeX index items:
        AWIP - All Words in Passage
        NWIV - NUmber of Words in Verse
        PNIP - Proper noun in passage
        ... more later
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with indexed items\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        verseRef = "!" + booknm + "!" + chapnm + " " + chapno + ":" + versno 
        line = line[5]
        if versn2 == '1':
            verseString = topString + textColorString + line + endString
        else:
            verseString = '[' + versn2 + '] ' + textColorString + line + endString
        #print(booknm,chapnm,chapno,versno,versn2) 
        '''
        NWIV
        '''
        words = line.split()
        numWords = len(words)
        if DEBUG == 1:
            print(numWords,'Words in verse',versn2)
        extraString="\\index[NWIV]{" + str(numWords) + verseRef + "}"
        '''
        AWIP - here, Passage = verse
        '''
        verseDict = {}
        wordString = ''
        for word in words:
            word = removePunc(word)
            if word not in verseDict:
                verseDict[word] = 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef + "}"
            else:
                verseDict[word] = verseDict[word] + 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef 
                wordString = wordString + " (" + str(verseDict[word]) + ")}"
        '''
        PNIP - Proper names in passage
        '''
        propernameStrng = ''
        for item in verseDict:
            if item in properNameList:
                if item not in exceptions:
                    propernameStrng = propernameStrng + "\\index[PNIP]{" + item + verseRef + "}"

        builtVerseString = verseString + extraString + wordString  + propernameStrng
        print(builtVerseString)
    return linesWith18Words, linesWith13Words

##########
def showWordsByFrequency(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words by Frequency}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def showWordsAlphabetically(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words Alphabetically}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[0], reverse=False)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def generateWordStatisticsForChapter(label,verses):
    '''
    Need to simplify this code! ONLY count number of words, number of unique 
    words, number of itallic words, number of unique itallic words
    
    verseDictionary, rest for each verse
    chapterDictionary
    '''
    global properNameList
    wordCount = 0
    emphasizedwordCount = 0
    wordDict = {}
    emphasizedwordDict = {}
    verseString = ''
    A = re.findall(r'\d+', label)
    A = " ".join(map(str, A))
    digit = int(A)

    word1 = " ".join(re.findall("[a-zA-Z]+", label))
    
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('%%%%% Word Statistics\n%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    
    print('\\normalsize')

    verseStats = ["\\subsection{Chapter Word Statistics}",
                  "\n",
                  "%%%%%%%%%%",
                  "%%%%%%%%%%",
                  " ",
                  "\\begin{center}",
                  "\\begin{longtable}{l|c|c|c|c}",
                  "\\caption[Stats for "+word1+" "+str(digit) +"]{Stats for "+word1+" "+str(digit) +"} \\label{table:Stats for "+word1+" "+str(digit) +"} \\\ ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endfirsthead",
                  " ",
                  "\multicolumn{5}{c}",
                  "{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\  ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endhead",
                  " ",
                  "\\hline \\multicolumn{5}{|r|}{{Continued if needed}} \\\ \\hline",
                  "\\endfoot ",]

    '''
    This section prints JUST the Bible text in a LaTeX format with a textcolor encoding added
    Original Code
    '''
    chapterDict = {}
    chapwordWords = 0
    chapterUniqueWords = 0
    numberChapterUniqueItallics = 0
    numberChapterItallics = 0
    verseNo = 0
    for line in verses:
        verseNo += 1
        verseDict = {}
#        print(line)
        words = removePunc(line[5])
        wordcount = 0
        uniqueWords = 0        
        '''
        Process for verse
        '''
        for word in words.split():
            wordcount += 1
            if word not in verseDict:
                verseDict[word] = 1
                uniqueWords += 1
            else:
                verseDict[word] = verseDict[word] + 1
        numberUniqueItallics = 0
        numberItallics = 0
        for item in verseDict:
            if "\emph{" in item:
                numberUniqueItallics += 1
                numberItallics = numberItallics + verseDict[item]                
        '''
        Process for chapter
        '''                
        for word in words.split():
            chapwordWords += 1
            if word not in chapterDict:
                chapterDict[word] = 1
                chapterUniqueWords += 1
            else:
                chapterDict[word] = chapterDict[word] + 1
        sequence = str(verseNo),str(wordcount),str(uniqueWords),str(numberItallics),str(numberUniqueItallics)+'\\\ \hline'
        S = ' & '.join(sequence)
        verseStats.append(S)
    for item in chapterDict:
        if "\emph{" in item:
            numberChapterUniqueItallics +=1
            numberChapterItallics = numberChapterItallics + chapterDict[item]

    '''
    Finish Up!
    '''
#    print ("\n",wordCount,"words in passage, of which",len(wordDict),"are unique")
#    print (emphasizedwordCount,'words in ital, of which',len(emphasizedwordDict),'are unique')
    print ("\n\n")
    verseStats.append("\hline \hline")
    sequence = 'Total',str(chapwordWords),str(chapterUniqueWords),str(numberChapterItallics),str(numberChapterUniqueItallics)
    S = ' & '.join(sequence)
    verseStats.append(S)
    for item in verseStats:
        print (item)
    print ("\n\n")
    print ("\\end{longtable}")
    print ("\\end{center}\n")
#    print (chapwordWords,"words in passage, of which",chapterUniqueWords,"are unique")
#    print (numberChapterItallics,'words in ital, of which',numberChapterUniqueItallics,'are unique')
#    print ("\n\n")
#
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
    
    return chapterDict

##########
def getWordLengthsInChapter(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    wordLengthDictionary = {}
    for item in dictionary:
        if "\emph{" in item:
            tempitem = item.replace("\emph{",'')
            tempitem = tempitem.replace('}','')
            if len(tempitem) in wordLengthDictionary:
                wordLengthDictionary[len(tempitem)] = wordLengthDictionary[len(tempitem)] + ', ' + item
            else:
                    wordLengthDictionary[len(tempitem)] = item
        else:
            if len(item) in wordLengthDictionary:
                wordLengthDictionary[len(item)] = wordLengthDictionary[len(item)] + ', ' + item
            else:
                wordLengthDictionary[len(item)] = item
    wordLengthDictionary = sorted(wordLengthDictionary.items(), key=lambda x:x[0], reverse=False)

    print('\\subsection{Word Lengths in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"]{Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in wordLengthDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")

##########
def makeVersesWith13WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 13 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 13 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########
          
##########
def makeVersesWith18WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 18 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 18 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########

def countLettersInVerses(arg1,arg2):
    for item in arg2:
        letterDict = {}
        for item2 in item[5]:
            if "\emph" in item2:
                item2 = item2.replace('\emph','')
            if item2 != ' ':
                if item2 in letterDict:
                    letterDict[item2] = letterDict[item2] + 1
                else:
                    letterDict[item2] =  1
            print ("for verse",item[2],":",letterDict)

def ProcessVerses(arg1,arg2):
    ''' 
    arg1 is dictionary name and arg2 is the dictionary
    '''
#    saveToPickleFile(arg1,arg2)  
    linesWith18words, linesWith13words = generatedLatexFormattedText(arg2)         # format chapter in Latex
    #linesWith13words is for use later ... 
    chapterDict = generateWordStatisticsForChapter(arg1,arg2)   # do chapter word statistics
    showWordsByFrequency(chapterDict,arg1)                      # show words by frequency
    showWordsAlphabetically(chapterDict,arg1)                   # show words alphabetically
    getWordLengthsInChapter(chapterDict,arg1)                   # show word lengths in chapter
    
    if len(linesWith13words) > 0:
        #print(linesWith13words)
        makeVersesWith13WordsTable(linesWith13words,arg1)
        
    if len(linesWith18words) > 0:
        #print(linesWith13words)
        makeVersesWith18WordsTable(linesWith18words,arg1)
        
    ''' In Development
    countLettersInVerses(arg1,arg2)
    '''



In [3]:
path = 'F:\python'
os.chdir(path)
print (os.getcwd())

F:\python


In [4]:
# %load NEW15OT-MakeConcordance-Ezra.py
"""
Created on Sun Jan 01 07:52:18 2017

@author: keith
"""


import os
import pickle
import string

from BibleWordList2 import *

BUILDAWIP = 1
BUILDNWIV = 1
BUILDPNIP = 1
BUILDNWIC = 1
BUILDNLIW = 1
BUILDBPIP = 1

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1
REBUILDDICTIONARYFROMPICKLEFILE = 0





Ezra01 = [
    ['Ezra','Ezr','01','001','1',"Now in the first year of Cyrus king of Persia, that the word of the LORD by the mouth of Jeremiah might be fulfilled, the LORD stirred up the spirit of Cyrus king of Persia, that he made a proclamation throughout all his kingdom, and \emph{put} \emph{it} also in writing, saying,"],
    ['Ezra','Ezr','01','002','2',"Thus saith Cyrus king of Persia, The LORD God of heaven hath given me all the kingdoms of the earth; and he hath charged me to build him an house at Jerusalem, which \emph{is} in Judah."],
    ['Ezra','Ezr','01','003','3',"Who \emph{is} \emph{there} among you of all his people? his God be with him, and let him go up to Jerusalem, which \emph{is} in Judah, and build the house of the LORD God of Israel, (he \emph{is} the God,) which \emph{is} in Jerusalem."],
    ['Ezra','Ezr','01','004','4',"And whosoever remaineth in any place where he sojourneth, let the men of his place help him with silver, and with gold, and with goods, and with beasts, beside the freewill offering for the house of God that \emph{is} in Jerusalem."],
    ['Ezra','Ezr','01','005','5',"Then rose up the chief of the fathers of Judah and Benjamin, and the priests, and the Levites, with all \emph{them} whose spirit God had raised, to go up to build the house of the LORD which \emph{is} in Jerusalem."],
    ['Ezra','Ezr','01','006','6',"And all they that \emph{were} about them strengthened their hands with vessels of silver, with gold, with goods, and with beasts, and with precious things, beside all \emph{that} was willingly offered."],
    ['Ezra','Ezr','01','007','7',"Also Cyrus the king brought forth the vessels of the house of the LORD, which Nebuchadnezzar had brought forth out of Jerusalem, and had put them in the house of his gods;"],
    ['Ezra','Ezr','01','008','8',"Even those did Cyrus king of Persia bring forth by the hand of Mithredath the treasurer, and numbered them unto Sheshbazzar, the prince of Judah."],
    ['Ezra','Ezr','01','009','9',"And this \emph{is} the number of them: thirty chargers of gold, a thousand chargers of silver, nine and twenty knives,"],
    ['Ezra','Ezr','01','010','10',"Thirty basons of gold, silver basons of a second \emph{sort} four hundred and ten, \emph{and} other vessels a thousand."],
    ['Ezra','Ezr','01','011','11',"All the vessels of gold and of silver \emph{were} five thousand and four hundred. All \emph{these} did Sheshbazzar bring up with \emph{them} \emph{of} the captivity that were brought up from Babylon unto Jerusalem."]
]

Ezra02 = [['Ezra','Ezr','02','001','1',"Now these \emph{are} the children of the province that went up out of the captivity, of those which had been carried away, whom Nebuchadnezzar the king of Babylon had carried away unto Babylon, and came again unto Jerusalem and Judah, every one unto his city;"],
['Ezra','Ezr','02','002','2',"Which came with Zerubbabel: Jeshua, Nehemiah, Seraiah, Reelaiah, Mordecai, Bilshan, Mispar, Bigvai, Rehum, Baanah. The number of the men of the people of Israel:"],
['Ezra','Ezr','02','003','3',"The children of Parosh, two thousand an hundred seventy and two."],
['Ezra','Ezr','02','004','4',"The children of Shephatiah, three hundred seventy and two."],
['Ezra','Ezr','02','005','5',"The children of Arah, seven hundred seventy and five."],
['Ezra','Ezr','02','006','6',"The children of Pahath-moab, of the children of Jeshua \emph{and} Joab, two thousand eight hundred and twelve."],
['Ezra','Ezr','02','007','7',"The children of Elam, a thousand two hundred fifty and four."],
['Ezra','Ezr','02','008','8',"The children of Zattu, nine hundred forty and five."],
['Ezra','Ezr','02','009','9',"The children of Zaccai, seven hundred and threescore."],
['Ezra','Ezr','02','010','10',"The children of Bani, six hundred forty and two."],
['Ezra','Ezr','02','011','11',"The children of Bebai, six hundred twenty and three."],
['Ezra','Ezr','02','012','12',"The children of Azgad, a thousand two hundred twenty and two."],
['Ezra','Ezr','02','013','13',"The children of Adonikam, six hundred sixty and six."],
['Ezra','Ezr','02','014','14',"The children of Bigvai, two thousand fifty and six."],
['Ezra','Ezr','02','015','15',"The children of Adin, four hundred fifty and four."],
['Ezra','Ezr','02','016','16',"The children of Ater of Hezekiah, ninety and eight."],
['Ezra','Ezr','02','017','17',"The children of Bezai, three hundred twenty and three."],
['Ezra','Ezr','02','018','18',"The children of Jorah, an hundred and twelve."],
['Ezra','Ezr','02','019','19',"The children of Hashum, two hundred twenty and three."],
['Ezra','Ezr','02','020','20',"The children of Gibbar, ninety and five."],
['Ezra','Ezr','02','021','21',"The children of Beth-lehem, an hundred twenty and three."],
['Ezra','Ezr','02','022','22',"The men of Netophah, fifty and six."],
['Ezra','Ezr','02','023','23',"The men of Anathoth, an hundred twenty and eight."],
['Ezra','Ezr','02','024','24',"The children of Azmaveth, forty and two."],
['Ezra','Ezr','02','025','25',"The children of Kirjath-arim, Chephirah, and Beeroth, seven hundred and forty and three."],
['Ezra','Ezr','02','026','26',"The children of Ramah and Gaba, six hundred twenty and one."],
['Ezra','Ezr','02','027','27',"The men of Michmas, an hundred twenty and two."],
['Ezra','Ezr','02','028','28',"The men of Beth-el and Ai, two hundred twenty and three."],
['Ezra','Ezr','02','029','29',"The children of Nebo, fifty and two."],
['Ezra','Ezr','02','030','30',"The children of Magbish, an hundred fifty and six."],
['Ezra','Ezr','02','031','31',"The children of the other Elam, a thousand two hundred fifty and four."],
['Ezra','Ezr','02','032','32',"The children of Harim, three hundred and twenty."],
['Ezra','Ezr','02','033','33',"The children of Lod, Hadid, and Ono, seven hundred twenty and five."],
['Ezra','Ezr','02','034','34',"The children of Jericho, three hundred forty and five."],
['Ezra','Ezr','02','035','35',"The children of Senaah, three thousand and six hundred and thirty."],
['Ezra','Ezr','02','036','36',"The priests: the children of Jedaiah, of the house of Jeshua, nine hundred seventy and three."],
['Ezra','Ezr','02','037','37',"The children of Immer, a thousand fifty and two."],
['Ezra','Ezr','02','038','38',"The children of Pashur, a thousand two hundred forty and seven."],
['Ezra','Ezr','02','039','39',"The children of Harim, a thousand and seventeen."],
['Ezra','Ezr','02','040','40',"The Levites: the children of Jeshua and Kadmiel, of the children of Hodaviah, seventy and four."],
['Ezra','Ezr','02','041','41',"The singers: the children of Asaph, an hundred twenty and eight."],
['Ezra','Ezr','02','042','42',"The children of the porters: the children of Shallum, the children of Ater, the children of Talmon, the children of Akkub, the children of Hatita, the children of Shobai, \emph{in} all an hundred thirty and nine."],
['Ezra','Ezr','02','043','43',"The Nethinims: the children of Ziha, the children of Hasupha, the children of Tabbaoth,"],
['Ezra','Ezr','02','044','44',"The children of Keros, the children of Siaha, the children of Padon,"],
['Ezra','Ezr','02','045','45',"The children of Lebanah, the children of Hagabah, the children of Akkub,"],
['Ezra','Ezr','02','046','46',"The children of Hagab, the children of Shalmai, the children of Hanan,"],
['Ezra','Ezr','02','047','47',"The children of Giddel, the children of Gahar, the children of Reaiah,"],
['Ezra','Ezr','02','048','48',"The children of Rezin, the children of Nekoda, the children of Gazzam,"],
['Ezra','Ezr','02','049','49',"The children of Uzza, the children of Paseah, the children of Besai,"],
['Ezra','Ezr','02','050','50',"The children of Asnah, the children of Mehunim, the children of Nephusim,"],
['Ezra','Ezr','02','051','51',"The children of Bakbuk, the children of Hakupha, the children of Harhur,"],
['Ezra','Ezr','02','052','52',"The children of Bazluth, the children of Mehida, the children of Harsha,"],
['Ezra','Ezr','02','053','53',"The children of Barkos, the children of Sisera, the children of Thamah,"],
['Ezra','Ezr','02','054','54',"The children of Neziah, the children of Hatipha."],
['Ezra','Ezr','02','055','55',"The children of Solomon's servants: the children of Sotai, the children of Sophereth, the children of Peruda,"],
['Ezra','Ezr','02','056','56',"The children of Jaalah, the children of Darkon, the children of Giddel,"],
['Ezra','Ezr','02','057','57',"The children of Shephatiah, the children of Hattil, the children of Pochereth of Zebaim, the children of Ami."],
['Ezra','Ezr','02','058','58',"All the Nethinims, and the children of Solomon's servants, \emph{were} three hundred ninety and two."],
['Ezra','Ezr','02','059','59',"And these \emph{were} they which went up from Tel-melah, Tel-harsa, Cherub, Addan, \emph{and} Immer: but they could not shew their father's house, and their seed, whether they \emph{were} of Israel:"],
['Ezra','Ezr','02','060','60',"The children of Delaiah, the children of Tobiah, the children of Nekoda, six hundred fifty and two."],
['Ezra','Ezr','02','061','61',"And of the children of the priests: the children of Habaiah, the children of Koz, the children of Barzillai; which took a wife of the daughters of Barzillai the Gileadite, and was called after their name:"],
['Ezra','Ezr','02','062','62',"These sought their register \emph{among} those that were reckoned by genealogy, but they were not found: therefore were they, as polluted, put from the priesthood."],
['Ezra','Ezr','02','063','63',"And the Tirshatha said unto them, that they should not eat of the most holy things, till there stood up a priest with Urim and with Thummim."],
['Ezra','Ezr','02','064','64',"The whole congregation together \emph{was} forty and two thousand three hundred \emph{and} threescore,"],
['Ezra','Ezr','02','065','65',"Beside their servants and their maids, of whom \emph{there} \emph{were} seven thousand three hundred thirty and seven: and \emph{there} \emph{were} among them two hundred singing men and singing women."],
['Ezra','Ezr','02','066','66',"Their horses \emph{were} seven hundred thirty and six; their mules, two hundred forty and five;"],
['Ezra','Ezr','02','067','67',"Their camels, four hundred thirty and five; \emph{their} asses, six thousand seven hundred and twenty."],
['Ezra','Ezr','02','068','68',"And \emph{some} of the chief of the fathers, when they came to the house of the LORD which \emph{is} at Jerusalem, offered freely for the house of God to set it up in his place:"],
['Ezra','Ezr','02','069','69',"They gave after their ability unto the treasure of the work threescore and one thousand drams of gold, and five thousand pound of silver, and one hundred priests' garments."],
['Ezra','Ezr','02','070','70',"So the priests, and the Levites, and \emph{some} of the people, and the singers, and the porters, and the Nethinims, dwelt in their cities, and all Israel in their cities."]
]

Ezra03 = [['Ezra','Ezr','03','001','1',"And when the seventh month was come, and the children of Israel \emph{were} in the cities, the people gathered themselves together as one man to Jerusalem."],
    ['Ezra','Ezr','03','002','2',"Then stood up Jeshua the son of Jozadak, and his brethren the priests, and Zerubbabel the son of Shealtiel, and his brethren, and builded the altar of the God of Israel, to offer burnt offerings thereon, as \emph{it} \emph{is} written in the law of Moses the man of God."],
    ['Ezra','Ezr','03','003','3',"And they set the altar upon his bases; for fear \emph{was} upon them because of the people of those countries: and they offered burnt offerings thereon unto the LORD, \emph{even} burnt offerings morning and evening."],
    ['Ezra','Ezr','03','004','4',"They kept also the feast of tabernacles, as \emph{it} \emph{is} written, and \emph{offered} the daily burnt offerings by number, according to the custom, as the duty of every day required;"],
    ['Ezra','Ezr','03','005','5',"And afterward \emph{offered} the continual burnt offering, both of the new moons, and of all the set feasts of the LORD that were consecrated, and of every one that willingly offered a freewill offering unto the LORD."],
    ['Ezra','Ezr','03','006','6',"From the first day of the seventh month began they to offer burnt offerings unto the LORD. But the foundation of the temple of the LORD was not \emph{yet} laid."],
    ['Ezra','Ezr','03','007','7',"They gave money also unto the masons, and to the carpenters; and meat, and drink, and oil, unto them of Zidon, and to them of Tyre, to bring cedar trees from Lebanon to the sea of Joppa, according to the grant that they had of Cyrus king of Persia."],
    ['Ezra','Ezr','03','008','8',"Now in the second year of their coming unto the house of God at Jerusalem, in the second month, began Zerubbabel the son of Shealtiel, and Jeshua the son of Jozadak, and the remnant of their brethren the priests and the Levites, and all they that were come out of the captivity unto Jerusalem; and appointed the Levites, from twenty years old and upward, to set forward the work of the house of the LORD."],
    ['Ezra','Ezr','03','009','9',"Then stood Jeshua \emph{with} his sons and his brethren, Kadmiel and his sons, the sons of Judah, together, to set forward the workmen in the house of God: the sons of Henadad, \emph{with} their sons and their brethren the Levites."],
    ['Ezra','Ezr','03','010','10',"And when the builders laid the foundation of the temple of the LORD, they set the priests in their apparel with trumpets, and the Levites the sons of Asaph with cymbals, to praise the LORD, after the ordinance of David king of Israel."],
    ['Ezra','Ezr','03','011','11',"And they sang together by course in praising and giving thanks unto the LORD; because \emph{he} \emph{is} good, for his mercy \emph{endureth} for ever toward Israel. And all the people shouted with a great shout, when they praised the LORD, because the foundation of the house of the LORD was laid."],
    ['Ezra','Ezr','03','012','12',"But many of the priests and Levites and chief of the fathers, \emph{who} \emph{were} ancient men, that had seen the first house, when the foundation of this house was laid before their eyes, wept with a loud voice; and many shouted aloud for joy:"],
    ['Ezra','Ezr','03','013','13',"So that the people could not discern the noise of the shout of joy from the noise of the weeping of the people: for the people shouted with a loud shout, and the noise was heard afar off."]
]

Ezra04 = [['Ezra','Ezr','04','001','1',"Now when the adversaries of Judah and Benjamin heard that the children of the captivity builded the temple unto the LORD God of Israel;"],
    ['Ezra','Ezr','04','002','2',"Then they came to Zerubbabel, and to the chief of the fathers, and said unto them, Let us build with you: for we seek your God, as ye \emph{do}; and we do sacrifice unto him since the days of Esar-haddon king of Assur, which brought us up hither."],
    ['Ezra','Ezr','04','003','3',"But Zerubbabel, and Jeshua, and the rest of the chief of the fathers of Israel, said unto them, Ye have nothing to do with us to build an house unto our God; but we ourselves together will build unto the LORD God of Israel, as king Cyrus the king of Persia hath commanded us."],
    ['Ezra','Ezr','04','004','4',"Then the people of the land weakened the hands of the people of Judah, and troubled them in building,"],
    ['Ezra','Ezr','04','005','5',"And hired counsellors against them, to frustrate their purpose, all the days of Cyrus king of Persia, even until the reign of Darius king of Persia."],
    ['Ezra','Ezr','04','006','6',"And in the reign of Ahasuerus, in the beginning of his reign, wrote they \emph{unto} \emph{him} an accusation against the inhabitants of Judah and Jerusalem."],
    ['Ezra','Ezr','04','007','7',"And in the days of Artaxerxes wrote Bishlam, Mithredath, Tabeel, and the rest of their companions, unto Artaxerxes king of Persia; and the writing of the letter \emph{was} written in the Syrian tongue, and interpreted in the Syrian tongue."],
    ['Ezra','Ezr','04','008','8',"Rehum the chancellor and Shimshai the scribe wrote a letter against Jerusalem to Artaxerxes the king in this sort:"],
    ['Ezra','Ezr','04','009','9',"Then \emph{wrote} Rehum the chancellor, and Shimshai the scribe, and the rest of their companions; the Dinaites, the Apharsathchites, the Tarpelites, the Apharsites, the Archevites, the Babylonians, the Susanchites, the Dehavites, \emph{and} the Elamites,"],
    ['Ezra','Ezr','04','010','10',"And the rest of the nations whom the great and noble Asnappar brought over, and set in the cities of Samaria, and the rest \emph{that} \emph{are} on this side the river, and at such a time."],
    ['Ezra','Ezr','04','011','11',"This \emph{is} the copy of the letter that they sent unto him, \emph{even} unto Artaxerxes the king; Thy servants the men on this side the river, and at such a time."],
    ['Ezra','Ezr','04','012','12',"Be it known unto the king, that the Jews which came up from thee to us are come unto Jerusalem, building the rebellious and the bad city, and have set up the walls \emph{thereof}, and joined the foundations."],
    ['Ezra','Ezr','04','013','13',"Be it known now unto the king, that, if this city be builded, and the walls set up \emph{again,} \emph{then} will they not pay toll, tribute, and custom, and \emph{so} thou shalt endamage the revenue of the kings."],
    ['Ezra','Ezr','04','014','14',"Now because we have maintenance from \emph{the} \emph{king's} palace, and it was not meet for us to see the king's dishonour, therefore have we sent and certified the king;"],
    ['Ezra','Ezr','04','015','15',"That search may be made in the book of the records of thy fathers: so shalt thou find in the book of the records, and know that this city \emph{is} a rebellious city, and hurtful unto kings and provinces, and that they have moved sedition within the same of old time: for which cause was this city destroyed."],
    ['Ezra','Ezr','04','016','16',"We certify the king that, if this city be builded \emph{again}, and the walls thereof set up, by this means thou shalt have no portion on this side the river."],
    ['Ezra','Ezr','04','017','17',"\emph{Then} sent the king an answer unto Rehum the chancellor, and \emph{to} Shimshai the scribe, and \emph{to} the rest of their companions that dwell in Samaria, and \emph{unto} the rest beyond the river, Peace, and at such a time."],
    ['Ezra','Ezr','04','018','18',"The letter which ye sent unto us hath been plainly read before me."],
    ['Ezra','Ezr','04','019','19',"And I commanded, and search hath been made, and it is found that this city of old time hath made insurrection against kings, and \emph{that} rebellion and sedition have been made therein."],
    ['Ezra','Ezr','04','020','20',"There have been mighty kings also over Jerusalem, which have ruled over all \emph{countries} beyond the river; and toll, tribute, and custom, was paid unto them."],
    ['Ezra','Ezr','04','021','21',"Give ye now commandment to cause these men to cease, and that this city be not builded, until \emph{another} commandment shall be given from me."],
    ['Ezra','Ezr','04','022','22',"Take heed now that ye fail not to do this: why should damage grow to the hurt of the kings?"],
    ['Ezra','Ezr','04','023','23',"Now when the copy of king Artaxerxes' letter \emph{was} read before Rehum, and Shimshai the scribe, and their companions, they went up in haste to Jerusalem unto the Jews, and made them to cease by force and power."],
    ['Ezra','Ezr','04','024','24',"Then ceased the work of the house of God which \emph{is} at Jerusalem. So it ceased unto the second year of the reign of Darius king of Persia."]
]


Ezra05 = [
    ['Ezra','Ezr','05','001','1',"Then the prophets, Haggai the prophet, and Zechariah the son of Iddo, prophesied unto the Jews that \emph{were} in Judah and Jerusalem in the name of the God of Israel, \emph{even} unto them."],
    ['Ezra','Ezr','05','002','2',"Then rose up Zerubbabel the son of Shealtiel, and Jeshua the son of Jozadak, and began to build the house of God which \emph{is} at Jerusalem: and with them \emph{were} the prophets of God helping them."],
    ['Ezra','Ezr','05','003','3',"At the same time came to them Tatnai, governor on this side the river, and Shethar-boznai, and their companions, and said thus unto them, Who hath commanded you to build this house, and to make up this wall?"],
    ['Ezra','Ezr','05','004','4',"Then said we unto them after this manner, What are the names of the men that make this building?"],
    ['Ezra','Ezr','05','005','5',"But the eye of their God was upon the elders of the Jews, that they could not cause them to cease, till the matter came to Darius: and then they returned answer by letter concerning this \emph{matter}."],
    ['Ezra','Ezr','05','006','6',"The copy of the letter that Tatnai, governor on this side the river, and Shethar-boznai, and his companions the Apharsachites, which \emph{were} on this side the river, sent unto Darius the king:"],
    ['Ezra','Ezr','05','007','7',"They sent a letter unto him, wherein was written thus; Unto Darius the king, all peace."],
    ['Ezra','Ezr','05','008','8',"Be it known unto the king, that we went into the province of Judea, to the house of the great God, which is builded with great stones, and timber is laid in the walls, and this work goeth fast on, and prospereth in their hands."],
    ['Ezra','Ezr','05','009','9',"Then asked we those elders, \emph{and} said unto them thus, Who commanded you to build this house, and to make up these walls?"],
    ['Ezra','Ezr','05','010','10',"We asked their names also, to certify thee, that we might write the names of the men that \emph{were} the chief of them."],
    ['Ezra','Ezr','05','011','11',"And thus they returned us answer, saying, We are the servants of the God of heaven and earth, and build the house that was builded these many years ago, which a great king of Israel builded and set up."],
    ['Ezra','Ezr','05','012','12',"But after that our fathers had provoked the God of heaven unto wrath, he gave them into the hand of Nebuchadnezzar the king of Babylon, the Chaldean, who destroyed this house, and carried the people away into Babylon."],
    ['Ezra','Ezr','05','013','13',"But in the first year of Cyrus the king of Babylon \emph{the} \emph{same} king Cyrus made a decree to build this house of God."],
    ['Ezra','Ezr','05','014','14',"And the vessels also of gold and silver of the house of God, which Nebuchadnezzar took out of the temple that \emph{was} in Jerusalem, and brought them into the temple of Babylon, those did Cyrus the king take out of the temple of Babylon, and they were delivered unto \emph{one}, whose name \emph{was} Sheshbazzar, whom he had made governor;"],
    ['Ezra','Ezr','05','015','15',"And said unto him, Take these vessels, go, carry them into the temple that \emph{is} in Jerusalem, and let the house of God be builded in his place."],
    ['Ezra','Ezr','05','016','16',"Then came the same Sheshbazzar, \emph{and} laid the foundation of the house of God which \emph{is} in Jerusalem: and since that time even until now hath it been in building, and \emph{yet} it is not finished."],
    ['Ezra','Ezr','05','017','17',"Now therefore, if \emph{it} \emph{seem} good to the king, let there be search made in the king's treasure house, which \emph{is} there at Babylon, whether it be \emph{so}, that a decree was made of Cyrus the king to build this house of God at Jerusalem, and let the king send his pleasure to us concerning this matter."]
]

Ezra06 = [['Ezra','Ezr','06','001','1',"Then Darius the king made a decree, and search was made in the house of the rolls, where the treasures were laid up in Babylon."],
['Ezra','Ezr','06','002','2',"And there was found at Achmetha, in the palace that \emph{is} in the province of the Medes, a roll, and therein \emph{was} a record thus written:"],
['Ezra','Ezr','06','003','3',"In the first year of Cyrus the king \emph{the} \emph{same} Cyrus the king made a decree \emph{concerning} the house of God at Jerusalem, Let the house be builded, the place where they offered sacrifices, and let the foundations thereof be strongly laid; the height thereof threescore cubits, \emph{and} the breadth thereof threescore cubits;"],
['Ezra','Ezr','06','004','4',"\emph{With} three rows of great stones, and a row of new timber: and let the expenses be given out of the king's house:"],
['Ezra','Ezr','06','005','5',"And also let the golden and silver vessels of the house of God, which Nebuchadnezzar took forth out of the temple which \emph{is} at Jerusalem, and brought unto Babylon, be restored, and brought again unto the temple which \emph{is} at Jerusalem, \emph{every} \emph{one} to his place, and place \emph{them} in the house of God."],
['Ezra','Ezr','06','006','6',"Now \emph{therefore}, Tatnai, governor beyond the river, Shethar-boznai, and your companions the Apharsachites, which \emph{are} beyond the river, be ye far from thence:"],
['Ezra','Ezr','06','007','7',"Let the work of this house of God alone; let the governor of the Jews and the elders of the Jews build this house of God in his place."],
['Ezra','Ezr','06','008','8',"Moreover I make a decree what ye shall do to the elders of these Jews for the building of this house of God: that of the king's goods, \emph{even} of the tribute beyond the river, forthwith expenses be given unto these men, that they be not hindered."],
['Ezra','Ezr','06','009','9',"And that which they have need of, both young bullocks, and rams, and lambs, for the burnt offerings of the God of heaven, wheat, salt, wine, and oil, according to the appointment of the priests which \emph{are} at Jerusalem, let it be given them day by day without fail:"],
['Ezra','Ezr','06','010','10',"That they may offer sacrifices of sweet savours unto the God of heaven, and pray for the life of the king, and of his sons."],
['Ezra','Ezr','06','011','11',"Also I have made a decree, that whosoever shall alter this word, let timber be pulled down from his house, and being set up, let him be hanged thereon; and let his house be made a dunghill for this."],
['Ezra','Ezr','06','012','12',"And the God that hath caused his name to dwell there destroy all kings and people, that shall put to their hand to alter \emph{and} to destroy this house of God which \emph{is} at Jerusalem. I Darius have made a decree; let it be done with speed."],
['Ezra','Ezr','06','013','13',"Then Tatnai, governor on this side the river, Shethar-boznai, and their companions, according to that which Darius the king had sent, so they did speedily."],
['Ezra','Ezr','06','014','14',"And the elders of the Jews builded, and they prospered through the prophesying of Haggai the prophet and Zechariah the son of Iddo. And they builded, and finished \emph{it}, according to the commandment of the God of Israel, and according to the commandment of Cyrus, and Darius, and Artaxerxes king of Persia."],
['Ezra','Ezr','06','015','15',"And this house was finished on the third day of the month Adar, which was in the sixth year of the reign of Darius the king."],
['Ezra','Ezr','06','016','16',"And the children of Israel, the priests, and the Levites, and the rest of the children of the captivity, kept the dedication of this house of God with joy,"],
['Ezra','Ezr','06','017','17',"And offered at the dedication of this house of God an hundred bullocks, two hundred rams, four hundred lambs; and for a sin offering for all Israel, twelve he goats, according to the number of the tribes of Israel."],
['Ezra','Ezr','06','018','18',"And they set the priests in their divisions, and the Levites in their courses, for the service of God, which \emph{is} at Jerusalem; as it is written in the book of Moses."],
['Ezra','Ezr','06','019','19',"And the children of the captivity kept the passover upon the fourteenth \emph{day} of the first month."],
['Ezra','Ezr','06','020','20',"For the priests and the Levites were purified together, all of them \emph{were} pure, and killed the passover for all the children of the captivity, and for their brethren the priests, and for themselves."],
['Ezra','Ezr','06','021','21',"And the children of Israel, which were come again out of captivity, and all such as had separated themselves unto them from the filthiness of the heathen of the land, to seek the LORD God of Israel, did eat,"],
['Ezra','Ezr','06','022','22',"And kept the feast of unleavened bread seven days with joy: for the LORD had made them joyful, and turned the heart of the king of Assyria unto them, to strengthen their hands in the work of the house of God, the God of Israel."]
        ]

Ezra07 = [['Ezra','Ezr','07','001','1',"Now after these things, in the reign of Artaxerxes king of Persia, Ezra the son of Seraiah, the son of Azariah, the son of Hilkiah,"],
    ['Ezra','Ezr','07','002','2',"The son of Shallum, the son of Zadok, the son of Ahitub,"],
    ['Ezra','Ezr','07','003','3',"The son of Amariah, the son of Azariah, the son of Meraioth,"],
    ['Ezra','Ezr','07','004','4',"The son of Zerahiah, the son of Uzzi, the son of Bukki,"],
    ['Ezra','Ezr','07','005','5',"The son of Abishua, the son of Phinehas, the son of Eleazar, the son of Aaron the chief priest:"],
    ['Ezra','Ezr','07','006','6',"This Ezra went up from Babylon; and he \emph{was} a ready scribe in the law of Moses, which the LORD God of Israel had given: and the king granted him all his request, according to the hand of the LORD his God upon him."],
    ['Ezra','Ezr','07','007','7',"And there went up \emph{some} of the children of Israel, and of the priests, and the Levites, and the singers, and the porters, and the Nethinims, unto Jerusalem, in the seventh year of Artaxerxes the king."],
    ['Ezra','Ezr','07','008','8',"And he came to Jerusalem in the fifth month, which \emph{was} in the seventh year of the king."],
    ['Ezra','Ezr','07','009','9',"For upon the first \emph{day} of the first month began he to go up from Babylon, and on the first \emph{day} of the fifth month came he to Jerusalem, according to the good hand of his God upon him."],
    ['Ezra','Ezr','07','010','10',"For Ezra had prepared his heart to seek the law of the LORD, and to do \emph{it}, and to teach in Israel statutes and judgments."],
    ['Ezra','Ezr','07','011','11',"Now this \emph{is} the copy of the letter that the king Artaxerxes gave unto Ezra the priest, the scribe, \emph{even} a scribe of the words of the commandments of the LORD, and of his statutes to Israel."],
    ['Ezra','Ezr','07','012','12',"Artaxerxes, king of kings, unto Ezra the priest, a scribe of the law of the God of heaven, perfect \emph{peace}, and at such a time."],
    ['Ezra','Ezr','07','013','13',"I make a decree, that all they of the people of Israel, and \emph{of} his priests and Levites, in my realm, which are minded of their own freewill to go up to Jerusalem, go with thee."],
    ['Ezra','Ezr','07','014','14',"Forasmuch as thou art sent of the king, and of his seven counsellors, to enquire concerning Judah and Jerusalem, according to the law of thy God which \emph{is} in thine hand;"],
    ['Ezra','Ezr','07','015','15',"And to carry the silver and gold, which the king and his counsellors have freely offered unto the God of Israel, whose habitation \emph{is} in Jerusalem,"],
    ['Ezra','Ezr','07','016','16',"And all the silver and gold that thou canst find in all the province of Babylon, with the freewill offering of the people, and of the priests, offering willingly for the house of their God which \emph{is} in Jerusalem:"],
    ['Ezra','Ezr','07','017','17',"That thou mayest buy speedily with this money bullocks, rams, lambs, with their meat offerings and their drink offerings, and offer them upon the altar of the house of your God which \emph{is} in Jerusalem."],
    ['Ezra','Ezr','07','018','18',"And whatsoever shall seem good to thee, and to thy brethren, to do with the rest of the silver and the gold, that do after the will of your God."],
    ['Ezra','Ezr','07','019','19',"The vessels also that are given thee for the service of the house of thy God, \emph{those} deliver thou before the God of Jerusalem."],
    ['Ezra','Ezr','07','020','20',"And whatsoever more shall be needful for the house of thy God, which thou shalt have occasion to bestow, bestow \emph{it} out of the king's treasure house."],
    ['Ezra','Ezr','07','021','21',"And I, \emph{even} I Artaxerxes the king, do make a decree to all the treasurers which \emph{are} beyond the river, that whatsoever Ezra the priest, the scribe of the law of the God of heaven, shall require of you, it be done speedily,"],
    ['Ezra','Ezr','07','022','22',"Unto an hundred talents of silver, and to an hundred measures of wheat, and to an hundred baths of wine, and to an hundred baths of oil, and salt without prescribing \emph{how} \emph{much}."],
    ['Ezra','Ezr','07','023','23',"Whatsoever is commanded by the God of heaven, let it be diligently done for the house of the God of heaven: for why should there be wrath against the realm of the king and his sons?"],
    ['Ezra','Ezr','07','024','24',"Also we certify you, that touching any of the priests and Levites, singers, porters, Nethinims, or ministers of this house of God, it shall not be lawful to impose toll, tribute, or custom, upon them."],
    ['Ezra','Ezr','07','025','25',"And thou, Ezra, after the wisdom of thy God, that \emph{is} in thine hand, set magistrates and judges, which may judge all the people that \emph{are} beyond the river, all such as know the laws of thy God; and teach ye them that know \emph{them} not."],
    ['Ezra','Ezr','07','026','26',"And whosoever will not do the law of thy God, and the law of the king, let judgment be executed speedily upon him, whether \emph{it} \emph{be} unto death, or to banishment, or to confiscation of goods, or to imprisonment."],
    ['Ezra','Ezr','07','027','27',"Blessed \emph{be} the LORD God of our fathers, which hath put \emph{such} \emph{a} \emph{thing} as this in the king's heart, to beautify the house of the LORD which \emph{is} in Jerusalem:"],
    ['Ezra','Ezr','07','028','28',"And hath extended mercy unto me before the king, and his counsellors, and before all the king's mighty princes. And I was strengthened as the hand of the LORD my God \emph{was} upon me, and I gathered together out of Israel chief men to go up with me."]
]

Ezra08 = [
    ['Ezra','Ezr','08','001','1',"These \emph{are} now the chief of their fathers, and \emph{this} \emph{is} the genealogy of them that went up with me from Babylon, in the reign of Artaxerxes the king."],
['Ezra','Ezr','08','002','2',"Of the sons of Phinehas; Gershom: of the sons of Ithamar; Daniel: of the sons of David; Hattush."],
['Ezra','Ezr','08','003','3',"Of the sons of Shechaniah, of the sons of Pharosh; Zechariah: and with him were reckoned by genealogy of the males an hundred and fifty."],
['Ezra','Ezr','08','004','4',"Of the sons of Pahath-moab; Elihoenai the son of Zerahiah, and with him two hundred males."],
['Ezra','Ezr','08','005','5',"Of the sons of Shechaniah; the son of Jahaziel, and with him three hundred males."],
['Ezra','Ezr','08','006','6',"Of the sons also of Adin; Ebed the son of Jonathan, and with him fifty males."],
['Ezra','Ezr','08','007','7',"And of the sons of Elam; Jeshaiah the son of Athaliah, and with him seventy males."],
['Ezra','Ezr','08','008','8',"And of the sons of Shephatiah; Zebadiah the son of Michael, and with him fourscore males."],
['Ezra','Ezr','08','009','9',"Of the sons of Joab; Obadiah the son of Jehiel, and with him two hundred and eighteen males."],
['Ezra','Ezr','08','010','10',"And of the sons of Shelomith; the son of Josiphiah, and with him an hundred and threescore males."],
['Ezra','Ezr','08','011','11',"And of the sons of Bebai; Zechariah the son of Bebai, and with him twenty and eight males."],
['Ezra','Ezr','08','012','12',"And of the sons of Azgad; Johanan the son of Hakkatan, and with him an hundred and ten males."],
['Ezra','Ezr','08','013','13',"And of the last sons of Adonikam, whose names \emph{are} these, Eliphelet, Jeiel, and Shemaiah, and with them threescore males."],
['Ezra','Ezr','08','014','14',"Of the sons also of Bigvai; Uthai, and Zabbud, and with them seventy males."],
['Ezra','Ezr','08','015','15',"And I gathered them together to the river that runneth to Ahava; and there abode we in tents three days: and I viewed the people, and the priests, and found there none of the sons of Levi."],
['Ezra','Ezr','08','016','16',"Then sent I for Eliezer, for Ariel, for Shemaiah, and for Elnathan, and for Jarib, and for Elnathan, and for Nathan, and for Zechariah, and for Meshullam, chief men; also for Joiarib, and for Elnathan, men of understanding."],
['Ezra','Ezr','08','017','17',"And I sent them with commandment unto Iddo the chief at the place Casiphia, and I told them what they should say unto Iddo, \emph{and} to his brethren the Nethinims, at the place Casiphia, that they should bring unto us ministers for the house of our God."],
['Ezra','Ezr','08','018','18',"And by the good hand of our God upon us they brought us a man of understanding, of the sons of Mahli, the son of Levi, the son of Israel; and Sherebiah, with his sons and his brethren, eighteen;"],
['Ezra','Ezr','08','019','19',"And Hashabiah, and with him Jeshaiah of the sons of Merari, his brethren and their sons, twenty;"],
['Ezra','Ezr','08','020','20',"Also of the Nethinims, whom David and the princes had appointed for the service of the Levites, two hundred and twenty Nethinims: all of them were expressed by name."],
['Ezra','Ezr','08','021','21',"Then I proclaimed a fast there, at the river of Ahava, that we might afflict ourselves before our God, to seek of him a right way for us, and for our little ones, and for all our substance."],
['Ezra','Ezr','08','022','22',"For I was ashamed to require of the king a band of soldiers and horsemen to help us against the enemy in the way: because we had spoken unto the king, saying, The hand of our God \emph{is} upon all them for good that seek him; but his power and his wrath \emph{is} against all them that forsake him."],
['Ezra','Ezr','08','023','23',"So we fasted and besought our God for this: and he was intreated of us."],
['Ezra','Ezr','08','024','24',"Then I separated twelve of the chief of the priests, Sherebiah, Hashabiah, and ten of their brethren with them,"],
['Ezra','Ezr','08','025','25',"And weighed unto them the silver, and the gold, and the vessels, \emph{even} the offering of the house of our God, which the king, and his counsellors, and his lords, and all Israel \emph{there} present, had offered:"],
['Ezra','Ezr','08','026','26',"I even weighed unto their hand six hundred and fifty talents of silver, and silver vessels an hundred talents, \emph{and} of gold an hundred talents;"],
['Ezra','Ezr','08','027','27',"Also twenty basons of gold, of a thousand drams; and two vessels of fine copper, precious as gold."],
['Ezra','Ezr','08','028','28',"And I said unto them, Ye \emph{are} holy unto the LORD; the vessels \emph{are} holy also; and the silver and the gold \emph{are} a freewill offering unto the LORD God of your fathers."],
['Ezra','Ezr','08','029','29',"Watch ye, and keep \emph{them}, until ye weigh \emph{them} before the chief of the priests and the Levites, and chief of the fathers of Israel, at Jerusalem, in the chambers of the house of the LORD."],
['Ezra','Ezr','08','030','30',"So took the priests and the Levites the weight of the silver, and the gold, and the vessels, to bring \emph{them} to Jerusalem unto the house of our God."],
['Ezra','Ezr','08','031','31',"Then we departed from the river of Ahava on the twelfth \emph{day} of the first month, to go unto Jerusalem: and the hand of our God was upon us, and he delivered us from the hand of the enemy, and of such as lay in wait by the way."],
['Ezra','Ezr','08','032','32',"And we came to Jerusalem, and abode there three days."],
['Ezra','Ezr','08','033','33',"Now on the fourth day was the silver and the gold and the vessels weighed in the house of our God by the hand of Meremoth the son of Uriah the priest; and with him \emph{was} Eleazar the son of Phinehas; and with them \emph{was} Jozabad the son of Jeshua, and Noadiah the son of Binnui, Levites;"],
['Ezra','Ezr','08','034','34',"By number \emph{and} by weight of every one: and all the weight was written at that time."],
['Ezra','Ezr','08','035','35',"\emph{Also} the children of those that had been carried away, which were come out of the captivity, offered burnt offerings unto the God of Israel, twelve bullocks for all Israel, ninety and six rams, seventy and seven lambs, twelve he goats \emph{for} a sin offering: all \emph{this} \emph{was} a burnt offering unto the LORD."],
['Ezra','Ezr','08','036','36',"And they delivered the king's commissions unto the king's lieutenants, and to the governors on this side the river: and they furthered the people, and the house of God."]
]

Ezra09 = [
    ['Ezra','Ezr','09','001','1',"Now when these things were done, the princes came to me, saying, The people of Israel, and the priests, and the Levites, have not separated themselves from the people of the lands, \emph{doing} according to their abominations, \emph{even} of the Canaanites, the Hittites, the Perizzites, the Jebusites, the Ammonites, the Moabites, the Egyptians, and the Amorites."],
['Ezra','Ezr','09','002','2',"For they have taken of their daughters for themselves, and for their sons: so that the holy seed have mingled themselves with the people of \emph{those} lands: yea, the hand of the princes and rulers hath been chief in this trespass."],
['Ezra','Ezr','09','003','3',"And when I heard this thing, I rent my garment and my mantle, and plucked off the hair of my head and of my beard, and sat down astonied."],
['Ezra','Ezr','09','004','4',"Then were assembled unto me every one that trembled at the words of the God of Israel, because of the transgression of those that had been carried away; and I sat astonied until the evening sacrifice."],
['Ezra','Ezr','09','005','5',"And at the evening sacrifice I arose up from my heaviness; and having rent my garment and my mantle, I fell upon my knees, and spread out my hands unto the LORD my God,"],
['Ezra','Ezr','09','006','6',"And said, O my God, I am ashamed and blush to lift up my face to thee, my God: for our iniquities are increased over \emph{our} head, and our trespass is grown up unto the heavens."],
['Ezra','Ezr','09','007','7',"Since the days of our fathers \emph{have} we \emph{been} in a great trespass unto this day; and for our iniquities have we, our kings, \emph{and} our priests, been delivered into the hand of the kings of the lands, to the sword, to captivity, and to a spoil, and to confusion of face, as \emph{it} \emph{is} this day."],
['Ezra','Ezr','09','008','8',"And now for a little space grace hath been \emph{shewed} from the LORD our God, to leave us a remnant to escape, and to give us a nail in his holy place, that our God may lighten our eyes, and give us a little reviving in our bondage."],
['Ezra','Ezr','09','009','9',"For we \emph{were} bondmen; yet our God hath not forsaken us in our bondage, but hath extended mercy unto us in the sight of the kings of Persia, to give us a reviving, to set up the house of our God, and to repair the desolations thereof, and to give us a wall in Judah and in Jerusalem."],
['Ezra','Ezr','09','010','10',"And now, O our God, what shall we say after this? for we have forsaken thy commandments,"],
['Ezra','Ezr','09','011','11',"Which thou hast commanded by thy servants the prophets, saying, The land, unto which ye go to possess it, is an unclean land with the filthiness of the people of the lands, with their abominations, which have filled it from one end to another with their uncleanness."],
['Ezra','Ezr','09','012','12',"Now therefore give not your daughters unto their sons, neither take their daughters unto your sons, nor seek their peace or their wealth for ever: that ye may be strong, and eat the good of the land, and leave \emph{it} for an inheritance to your children for ever."],
['Ezra','Ezr','09','013','13',"And after all that is come upon us for our evil deeds, and for our great trespass, seeing that thou our God hast punished us less than our iniquities \emph{deserve}, and hast given us \emph{such} deliverance as this;"],
['Ezra','Ezr','09','014','14',"Should we again break thy commandments, and join in affinity with the people of these abominations? wouldest not thou be angry with us till thou hadst consumed \emph{us}, so that \emph{there} \emph{should} \emph{be} no remnant nor escaping?"],
['Ezra','Ezr','09','015','15',"O LORD God of Israel, thou \emph{art} righteous: for we remain yet escaped, as \emph{it} \emph{is} this day: behold, we \emph{are} before thee in our trespasses: for we cannot stand before thee because of this."]
]

Ezra10 = [['Ezra','Ezr','10','001','1',"Now when Ezra had prayed, and when he had confessed, weeping and casting himself down before the house of God, there assembled unto him out of Israel a very great congregation of men and women and children: for the people wept very sore."],
['Ezra','Ezr','10','002','2',"And Shechaniah the son of Jehiel, \emph{one} of the sons of Elam, answered and said unto Ezra, We have trespassed against our God, and have taken strange wives of the people of the land: yet now there is hope in Israel concerning this thing."],
['Ezra','Ezr','10','003','3',"Now therefore let us make a covenant with our God to put away all the wives, and such as are born of them, according to the counsel of my lord, and of those that tremble at the commandment of our God; and let it be done according to the law."],
['Ezra','Ezr','10','004','4',"Arise; for \emph{this} matter \emph{belongeth} unto thee: we also \emph{will} \emph{be} with thee: be of good courage, and do \emph{it}."],
['Ezra','Ezr','10','005','5',"Then arose Ezra, and made the chief priests, the Levites, and all Israel, to swear that they should do according to this word. And they sware."],
['Ezra','Ezr','10','006','6',"Then Ezra rose up from before the house of God, and went into the chamber of Johanan the son of Eliashib: and \emph{when} he came thither, he did eat no bread, nor drink water: for he mourned because of the transgression of them that had been carried away."],
['Ezra','Ezr','10','007','7',"And they made proclamation throughout Judah and Jerusalem unto all the children of the captivity, that they should gather themselves together unto Jerusalem;"],
['Ezra','Ezr','10','008','8',"And that whosoever would not come within three days, according to the counsel of the princes and the elders, all his substance should be forfeited, and himself separated from the congregation of those that had been carried away."],
['Ezra','Ezr','10','009','9',"Then all the men of Judah and Benjamin gathered themselves together unto Jerusalem within three days. It \emph{was} the ninth month, on the twentieth \emph{day} of the month; and all the people sat in the street of the house of God, trembling because of \emph{this} matter, and for the great rain."],
['Ezra','Ezr','10','010','10',"And Ezra the priest stood up, and said unto them, Ye have transgressed, and have taken strange wives, to increase the trespass of Israel."],
['Ezra','Ezr','10','011','11',"Now therefore make confession unto the LORD God of your fathers, and do his pleasure: and separate yourselves from the people of the land, and from the strange wives."],
['Ezra','Ezr','10','012','12',"Then all the congregation answered and said with a loud voice, As thou hast said, so must we do."],
['Ezra','Ezr','10','013','13',"But the people \emph{are} many, and \emph{it} \emph{is} a time of much rain, and we are not able to stand without, neither \emph{is} \emph{this} a work of one day or two: for we are many that have transgressed in this thing."],
['Ezra','Ezr','10','014','14',"Let now our rulers of all the congregation stand, and let all them which have taken strange wives in our cities come at appointed times, and with them the elders of every city, and the judges thereof, until the fierce wrath of our God for this matter be turned from us."],
['Ezra','Ezr','10','015','15',"Only Jonathan the son of Asahel and Jahaziah the son of Tikvah were employed about this \emph{matter}: and Meshullam and Shabbethai the Levite helped them."],
['Ezra','Ezr','10','016','16',"And the children of the captivity did so. And Ezra the priest, \emph{with} certain chief of the fathers, after the house of their fathers, and all of them by \emph{their} names, were separated, and sat down in the first day of the tenth month to examine the matter."],
['Ezra','Ezr','10','017','17',"And they made an end with all the men that had taken strange wives by the first day of the first month."],
['Ezra','Ezr','10','018','18',"And among the sons of the priests there were found that had taken strange wives: \emph{namely}, of the sons of Jeshua the son of Jozadak, and his brethren; Maaseiah, and Eliezer, and Jarib, and Gedaliah."],
['Ezra','Ezr','10','019','19',"And they gave their hands that they would put away their wives; and \emph{being} guilty, \emph{they} \emph{offered} a ram of the flock for their trespass."],
['Ezra','Ezr','10','020','20',"And of the sons of Immer; Hanani, and Zebadiah."],
['Ezra','Ezr','10','021','21',"And of the sons of Harim; Maaseiah, and Elijah, and Shemaiah, and Jehiel, and Uzziah."],
['Ezra','Ezr','10','022','22',"And of the sons of Pashur; Elioenai, Maaseiah, Ishmael, Nethaneel, Jozabad, and Elasah."],
['Ezra','Ezr','10','023','23',"Also of the Levites; Jozabad, and Shimei, and Kelaiah, (the same \emph{is} Kelita,) Pethahiah, Judah, and Eliezer."],
['Ezra','Ezr','10','024','24',"Of the singers also; Eliashib: and of the porters; Shallum, and Telem, and Uri."],
['Ezra','Ezr','10','025','25',"Moreover of Israel: of the sons of Parosh; Ramiah, and Jeziah, and Malchiah, and Miamin, and Eleazar, and Malchijah, and Benaiah."],
['Ezra','Ezr','10','026','26',"And of the sons of Elam; Mattaniah, Zechariah, and Jehiel, and Abdi, and Jeremoth, and Eliah."],
['Ezra','Ezr','10','027','27',"And of the sons of Zattu; Elioenai, Eliashib, Mattaniah, and Jeremoth, and Zabad, and Aziza."],
['Ezra','Ezr','10','028','28',"Of the sons also of Bebai; Jehohanan, Hananiah, Zabbai, \emph{and} Athlai."],
['Ezra','Ezr','10','029','29',"And of the sons of Bani; Meshullam, Malluch, and Adaiah, Jashub, and Sheal, and Ramoth."],
['Ezra','Ezr','10','030','30',"And of the sons of Pahath-moab; Adna, and Chelal, Benaiah, Maaseiah, Mattaniah, Bezaleel, and Binnui, and Manasseh."],
['Ezra','Ezr','10','031','31',"And \emph{of} the sons of Harim; Eliezer, Ishijah, Malchiah, Shemaiah, Shimeon,"],
['Ezra','Ezr','10','032','32',"Benjamin, Malluch, \emph{and} Shemariah."],
['Ezra','Ezr','10','033','33',"Of the sons of Hashum; Mattenai, Mattathah, Zabad, Eliphelet, Jeremai, Manasseh, \emph{and} Shimei."],
['Ezra','Ezr','10','034','34',"Of the sons of Bani; Maadai, Amram, and Uel,"],
['Ezra','Ezr','10','035','35',"Benaiah, Bedeiah, Chelluh,"],
['Ezra','Ezr','10','036','36',"Vaniah, Meremoth, Eliashib,"],
['Ezra','Ezr','10','037','37',"Mattaniah, Mattenai, and Jaasau,"],
['Ezra','Ezr','10','038','38',"And Bani, and Binnui, Shimei,"],
['Ezra','Ezr','10','039','39',"And Shelemiah, and Nathan, and Adaiah,"],
['Ezra','Ezr','10','040','40',"Machnadebai, Shashai, Sharai,"],
['Ezra','Ezr','10','041','41',"Azareel, and Shelemiah, Shemariah,"],
['Ezra','Ezr','10','042','42',"Shallum, Amariah, \emph{and} Joseph."],
['Ezra','Ezr','10','043','43',"Of the sons of Nebo; Jeiel, Mattithiah, Zabad, Zebina, Jadau, and Joel, Benaiah."],
['Ezra','Ezr','10','044','44',"All these had taken strange wives: and \emph{some} of them had wives by whom they had children."]
    
]

def main(): 
    ProcessVerses('Ezra10',Ezra10)

if __name__ == "__main__":
    main()



%%%%%%%%%%%%%%%%%%%%%%%%%%%
Chapter with Latex format
%%%%%%%%%%%%%%%%%%%%%%%%%%%


\footnote{\textcolor[cmyk]{0.99998,1,0,0}{\hyperlink{TOC}{Return to end of Table of Contents.}}}\textcolor[cmyk]{0.99998,1,0,0}{Now when Ezra had prayed, and when he had confessed, weeping and casting himself down before the house of God, there assembled unto him out of Israel a very great congregation of men and women and children: for the people wept very sore.}
[2] \textcolor[cmyk]{0.99998,1,0,0}{And Shechaniah the son of Jehiel, \emph{one} of the sons of Elam, answered and said unto Ezra, We have trespassed against our God, and have taken strange wives of the people of the land: yet now there is hope in Israel concerning this thing.}
[3] \textcolor[cmyk]{0.99998,1,0,0}{Now therefore let us make a covenant with our God to put away all the wives, and such as are born of them, according to the counsel of my lord, and of those that tremble at the commandment of our God; and let it be done according to

[14] \textcolor[cmyk]{0.99998,1,0,0}{Let now our rulers of all the congregation stand, and let all them which have taken strange wives in our cities come at appointed times, and with them the elders of every city, and the judges thereof, until the fierce wrath of our God for this matter be turned from us.}\index[NWIV]{51!Ezra!Ezr 10:014}\index[AWIP]{Let!Ezra!Ezr 10:014}\index[AWIP]{now!Ezra!Ezr 10:014}\index[AWIP]{our!Ezra!Ezr 10:014}\index[AWIP]{rulers!Ezra!Ezr 10:014}\index[AWIP]{of!Ezra!Ezr 10:014}\index[AWIP]{all!Ezra!Ezr 10:014}\index[AWIP]{the!Ezra!Ezr 10:014}\index[AWIP]{congregation!Ezra!Ezr 10:014}\index[AWIP]{stand!Ezra!Ezr 10:014}\index[AWIP]{and!Ezra!Ezr 10:014}\index[AWIP]{let!Ezra!Ezr 10:014}\index[AWIP]{all!Ezra!Ezr 10:014 (2)}\index[AWIP]{them!Ezra!Ezr 10:014}\index[AWIP]{which!Ezra!Ezr 10:014}\index[AWIP]{have!Ezra!Ezr 10:014}\index[AWIP]{taken!Ezra!Ezr 10:014}\index[AWIP]{strange!Ezra!Ezr 10:014}\index[AWIP]{wives!Ezra!Ezr 10:014}\index[AWIP]{in!Ezra!Ezr 10:014}\index

Adna & 1 \\ \hline
Chelal & 1 \\ \hline
Bezaleel & 1 \\ \hline
\emph{of} & 1 \\ \hline
Ishijah & 1 \\ \hline
Shimeon & 1 \\ \hline
Hashum & 1 \\ \hline
Mattathah & 1 \\ \hline
Eliphelet & 1 \\ \hline
Jeremai & 1 \\ \hline
Maadai & 1 \\ \hline
Amram & 1 \\ \hline
Uel & 1 \\ \hline
Bedeiah & 1 \\ \hline
Chelluh & 1 \\ \hline
Vaniah & 1 \\ \hline
Meremoth & 1 \\ \hline
Jaasau & 1 \\ \hline
Nathan & 1 \\ \hline
Machnadebai & 1 \\ \hline
Shashai & 1 \\ \hline
Sharai & 1 \\ \hline
Azareel & 1 \\ \hline
Amariah & 1 \\ \hline
Joseph & 1 \\ \hline
Nebo & 1 \\ \hline
Jeiel & 1 \\ \hline
Mattithiah & 1 \\ \hline
Zebina & 1 \\ \hline
Jadau & 1 \\ \hline
Joel & 1 \\ \hline
All & 1 \\ \hline
these & 1 \\ \hline
\emph{some} & 1 \\ \hline
whom & 1 \\ \hline
\end{longtable}
\end{center}



\normalsize



\subsection{Words Alphabetically}
\scriptsize
\begin{center}
\begin{longtable}{l|r}
\caption[Word Alphabetically in Ezra 10]{Word Alphabetically in Ezra 10} \label{table:WordsIn-Ezra-10} \\ 
\hline \mu